In [1]:
from IPython.display import Image 
%matplotlib inline

import torch
import torch.nn as nn

In [2]:
rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True)

In [3]:
w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0

In [4]:
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

In [5]:
print(f"W_xh 크기 : {w_xh.shape}")
print(f"W_hh 크기 : {w_hh.shape}")
print(f"b_xh 크기 : {b_xh.shape}")
print(f"b_hh 크기 : {b_hh.shape}")

W_xh 크기 : torch.Size([2, 5])
W_hh 크기 : torch.Size([2, 2])
b_xh 크기 : torch.Size([2])
b_hh 크기 : torch.Size([2])


In [6]:
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()

In [11]:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3,5)))

In [15]:
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t],(1,5))
    print(f"타임 스텝 {t} -> 입력 : {xt.numpy()}")
    ht = torch.matmul(xt, torch.transpose(w_xh, 0,1)) + b_xh


    if t > 0 :
        # 이전 값
        pass
    else:
        prev_h = torch.zeros((ht.shape))
   
    out = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    out = torch.tanh(out)
    out_man.append(out)
    print(f" 출력(수동) : {out.detach().numpy()}")
    print(f" RNN 출력 : {output[:,t].detach().numpy()}")
    print()



타임 스텝 0 -> 입력 : [[1. 1. 1. 1. 1.]]
 출력(수동) : [[-0.03897114  0.81699085]]
 RNN 출력 : [[-0.03897114  0.8169908 ]]

타임 스텝 1 -> 입력 : [[2. 2. 2. 2. 2.]]
 출력(수동) : [[0.6837227  0.99318826]]
 RNN 출력 : [[0.73590636 0.9796046 ]]

타임 스텝 2 -> 입력 : [[3. 3. 3. 3. 3.]]
 출력(수동) : [[0.9367865  0.99976814]]
 RNN 출력 : [[0.9803263 0.9994701]]



In [18]:
pip install dataset 

Note: you may need to restart the kernel to use updated packages.


감정 분석 모델 / IMDB 리뷰 긍/부정 분류 모델 

In [21]:
from datasets import load_dataset


dataset = load_dataset('imdb')

In [22]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [25]:
dataset['train']['text'].__len__()

25000

In [26]:
dataset['train']['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [27]:
dataset['train']['label'][0]

0

In [28]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [29]:
for text, label in zip(train_dataset['text'], train_dataset['label']):
    print(text, label)
    break

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [32]:
import re
from collections import Counter

token_counts = Counter(['a', 'a', 'b'])

In [33]:
token_counts

Counter({'a': 2, 'b': 1})

In [34]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized


In [35]:
tokenizer(text)

['i',
 'rented',
 'i',
 'am',
 'curious',
 'yellow',
 'from',
 'my',
 'video',
 'store',
 'because',
 'of',
 'all',
 'the',
 'controversy',
 'that',
 'surrounded',
 'it',
 'when',
 'it',
 'was',
 'first',
 'released',
 'in',
 '1967',
 'i',
 'also',
 'heard',
 'that',
 'at',
 'first',
 'it',
 'was',
 'seized',
 'by',
 'u',
 's',
 'customs',
 'if',
 'it',
 'ever',
 'tried',
 'to',
 'enter',
 'this',
 'country',
 'therefore',
 'being',
 'a',
 'fan',
 'of',
 'films',
 'considered',
 'controversial',
 'i',
 'really',
 'had',
 'to',
 'see',
 'this',
 'for',
 'myself',
 'the',
 'plot',
 'is',
 'centered',
 'around',
 'a',
 'young',
 'swedish',
 'drama',
 'student',
 'named',
 'lena',
 'who',
 'wants',
 'to',
 'learn',
 'everything',
 'she',
 'can',
 'about',
 'life',
 'in',
 'particular',
 'she',
 'wants',
 'to',
 'focus',
 'her',
 'attentions',
 'to',
 'making',
 'some',
 'sort',
 'of',
 'documentary',
 'on',
 'what',
 'the',
 'average',
 'swede',
 'thought',
 'about',
 'certain',
 'politica

In [ ]:
#IMDB 데이터셋에서 워드카운트 실행 / token은 한개의 단어를 의미하는 것과 같음 
token_counts = Counter()

for text, label in zip(train_dataset['text'], train_dataset['label']):
    #print(text, label)
    tokens = tokenizer(text)
    token_counts.update(tokens)

In [37]:
token_counts

Counter({'the': 336632,
         'and': 164102,
         'a': 163086,
         'of': 145850,
         'to': 135703,
         'is': 107312,
         'it': 96436,
         'in': 93954,
         'i': 87624,
         'this': 75964,
         'that': 73273,
         's': 63588,
         'was': 48196,
         'as': 46930,
         'for': 44332,
         'with': 44121,
         'movie': 44026,
         'but': 42612,
         'film': 40144,
         't': 34382,
         'you': 34260,
         'on': 34196,
         'not': 30625,
         'he': 30144,
         'are': 29432,
         'his': 29369,
         'have': 27725,
         'be': 26952,
         'one': 26788,
         'all': 23974,
         'at': 23507,
         'they': 22906,
         'by': 22541,
         'an': 21561,
         'who': 21435,
         'so': 20607,
         'from': 20494,
         'like': 20274,
         'there': 18856,
         'her': 18419,
         'or': 18001,
         'just': 17765,
         'about': 17372,
         'ou

In [39]:
len(token_counts)

75977

In [44]:
sorted(token_counts.items(), key=lambda x : x[1], reverse=True)

[('the', 336632),
 ('and', 164102),
 ('a', 163086),
 ('of', 145850),
 ('to', 135703),
 ('is', 107312),
 ('it', 96436),
 ('in', 93954),
 ('i', 87624),
 ('this', 75964),
 ('that', 73273),
 ('s', 63588),
 ('was', 48196),
 ('as', 46930),
 ('for', 44332),
 ('with', 44121),
 ('movie', 44026),
 ('but', 42612),
 ('film', 40144),
 ('t', 34382),
 ('you', 34260),
 ('on', 34196),
 ('not', 30625),
 ('he', 30144),
 ('are', 29432),
 ('his', 29369),
 ('have', 27725),
 ('be', 26952),
 ('one', 26788),
 ('all', 23974),
 ('at', 23507),
 ('they', 22906),
 ('by', 22541),
 ('an', 21561),
 ('who', 21435),
 ('so', 20607),
 ('from', 20494),
 ('like', 20274),
 ('there', 18856),
 ('her', 18419),
 ('or', 18001),
 ('just', 17765),
 ('about', 17372),
 ('out', 17109),
 ('if', 16799),
 ('has', 16786),
 ('what', 16165),
 ('some', 15744),
 ('good', 15140),
 ('can', 14670),
 ('more', 14244),
 ('she', 14223),
 ('when', 14178),
 ('very', 14063),
 ('up', 13291),
 ('time', 12723),
 ('no', 12714),
 ('even', 12645),
 ('my', 12

In [ ]:
import 

ModuleNotFoundError: No module named 'libtorchtext'